### 홀드아웃(Hold-out)
주어진 데이터를 랜덤하게 두개의 데이터로 구분하여 훈련/평가로 분류한 디 사용하는 방식이다. 
데이터가 충분한 경우, 훈련 데이터는 다시 훈련/검증용 데이터로 분류할 수 있다.

### 교차검증(K-fold Cross-Validation)
데이터를 무작위, 혹은 규칙을 설정하여 K개의 집단으로 구분한 뒤, 한 개의 데이터 집단을 검증용 데이터셋으로 활용한다. 이를 K번 반복하여 얻은 MSE값을 최종 성과지표로 활용된다.

데이터 셋을 K번 반복하여 학습하기 떄문에 시간이 오래 걸리는 단점이 있지만, 다양한 학습 데이터를 사용하는 관점에서 성능이 좋다는 장점이 있다.
